In [ ]:
reference_file = "\Users\kyn03\Downloads\서영님전달_이탈.xlsx"
check_file = "\Users\kyn03\Downloads\구매목록_최종(1018).xlsx"
output_file = "이탈자"
check_file에서 "상품명" 칼럼값이 "[10월 올영픽/보습윤광케어] 달바 화이트 트러플 퍼스트 스프레이 세럼 100ml + 100ml 기획"이거나 "[재구매 1위] 달바 화이트 트러플 퍼스트 스프레이 세럼 100ml"이면, 
가장 최근 구매 날짜를 확인하고 그 날짜가 이번 해 10월 5일 기준 181일 이전이면 output_file에 아이디 칼럼을 만들고 최근 날짜도 적어줘. 

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# 파일 경로 설정
reference_file = r"\Users\kyn03\Downloads\서영님전달_이탈.xlsx"
check_file = r"\Users\kyn03\Downloads\구매목록_최종(1018).xlsx"
output_file = r"\Users\kyn03\Downloads\이탈자.xlsx"

# 기준 날짜 설정 (2023년 10월 5일 기준, 181일 이전)
cutoff_date = datetime(2024, 10, 5) - timedelta(days=181)

# 데이터 읽기
check_df = pd.read_excel(check_file)

# 상품명 조건에 맞는 데이터 필터링
product_conditions = [
    "[10월 올영픽/보습윤광케어] 달바 화이트 트러플 퍼스트 스프레이 세럼 100ml + 100ml 기획",
    "[재구매 1위] 달바 화이트 트러플 퍼스트 스프레이 세럼 100ml"
]
filtered_check_df = check_df[check_df['상품명'].isin(product_conditions)]

# 날짜 변환 및 가장 최근 구매 날짜 확인
filtered_check_df['구매날짜'] = pd.to_datetime(filtered_check_df['구매날짜'], errors='coerce')
recent_purchases = filtered_check_df.groupby('아이디')['구매날짜'].max().reset_index()

# 기준 날짜 이전 구매자 필터링
recent_purchases_filtered = recent_purchases[recent_purchases['구매날짜'] <= cutoff_date]

# 결과 저장
output_df = recent_purchases_filtered.rename(columns={'구매날짜': '최근구매날짜'})
output_df.to_excel(output_file, index=False)

print(f"이탈자 목록이 {output_file} 파일에 저장되었습니다.")


이탈자 목록이 \Users\kyn03\Downloads\이탈자.csv 파일에 저장되었습니다.


C:\Users\kyn03\AppData\Local\Temp\ipykernel_24732\3383637181.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_check_df['리뷰 작성일'] = pd.to_datetime(filtered_check_df['리뷰 작성일'], errors='coerce')


In [ ]:
base_file에서 '리뷰어'칼럼 값 중 check_file '리뷰어 이름'에 없고 그 '날짜'칼럼 값이 이번 해 10월 5일 기준 181일 이전 값밖에 없는지 확인. 만약 그렇고, 만약 여러 개의 행이라면, base_file '날짜'가 최신인 것으로 한 행만 output_file에 base_file의 행을 append.
list_up file에서 '리뷰어 이름' 칼럼이 base_file의 '리뷰어' 칼럼과 같고 이번 해 10월 5일 기준 181일 이전 값밖에 없는지 확인. 만약 그렇다면, base_file의'날짜'칼럼 값이 최신인 것만 가져와서 행을 output_file에 append.
append하는 행이 같으면, 한 번만 하는 걸로.
base_file = r"\Users\kyn03\OneDrive\바탕 화면\project_file\달바_결측치처리_최종.xlsx" 
output_file = r"\Users\kyn03\Downloads\이탈자_리뷰.xlsx"
list_up = r"\Users\kyn03\Downloads\이탈자.xlsx"
check_file = r"\Users\kyn03\Downloads\구매목록_최종(1018).xlsx"

In [3]:
import pandas as pd
from datetime import datetime, timedelta

# 파일 경로 설정
base_file = r"\Users\kyn03\OneDrive\바탕 화면\project_file\달바_결측치처리_최종.xlsx"
output_file = r"\Users\kyn03\Downloads\이탈자_리뷰.xlsx"
list_up = r"\Users\kyn03\Downloads\이탈자.xlsx"
check_file = r"\Users\kyn03\Downloads\구매목록_최종(1018).xlsx"

# 기준 날짜 설정 (2024년 10월 5일 기준, 181일 이전)
cutoff_date = datetime(2024, 10, 5) - timedelta(days=181)

# 데이터 읽기
base_df = pd.read_excel(base_file)
check_df = pd.read_excel(check_file)
list_up_df = pd.read_excel(list_up)

# 날짜 변환
base_df['날짜'] = pd.to_datetime(base_df['날짜'], errors='coerce')
# check_df['날짜'] = pd.to_datetime(check_df['날짜'], errors='coerce')

# Step 1: base_file의 '리뷰어'가 check_file의 '리뷰어 이름'에 없고 날짜가 기준일 이전 값밖에 없는 경우
step1_candidates = base_df[~base_df['리뷰어'].isin(check_df['리뷰어 이름'])]
step1_valid = step1_candidates.groupby('리뷰어').filter(lambda x: x['날짜'].max() <= cutoff_date)
step1_df = step1_valid.loc[step1_valid.groupby('리뷰어')['날짜'].idxmax()]  # 최신 날짜만 선택

# Step 2: list_up의 '리뷰어 이름'이 base_file의 '리뷰어'와 같고 날짜가 기준일 이전 값밖에 없는 경우
merged_df = base_df.merge(list_up_df, left_on='리뷰어', right_on='리뷰어 이름', how='inner')
step2_valid = merged_df.groupby('리뷰어').filter(lambda x: x['날짜'].max() <= cutoff_date)
step2_df = step2_valid.loc[step2_valid.groupby('리뷰어')['날짜'].idxmax()]  # 최신 날짜만 선택

# 결과 합치기 및 중복 제거
output_df = pd.concat([step1_df, step2_df]).drop_duplicates()
output_df.to_excel(output_file, index=False)

print(f"결과가 {output_file}에 저장되었습니다.")


결과가 \Users\kyn03\Downloads\이탈자_리뷰.xlsx에 저장되었습니다.


In [2]:
import pandas as pd
from datetime import datetime, timedelta

# 파일 경로 설정
base_file = r"\Users\kyn03\OneDrive\바탕 화면\project_file\달바_결측치처리_최종.xlsx"
output_file = r"\Users\kyn03\Downloads\이탈자_리뷰_비공개_제거.xlsx"
list_up = r"\Users\kyn03\Downloads\이탈자.xlsx"

# 기준 날짜 설정 (2023년 10월 5일 기준, 181일 이전)
cutoff_date = datetime(2024, 10, 5) - timedelta(days=181)

# 데이터 읽기
base_df = pd.read_excel(base_file)
list_up_df = pd.read_excel(list_up)

# 날짜 변환
base_df['날짜'] = pd.to_datetime(base_df['날짜'], errors='coerce')

# Step: list_up의 '리뷰어 이름'과 base_file의 '리뷰어'가 같은 경우
merged_df = base_df.merge(list_up_df, left_on='리뷰어', right_on='리뷰어 이름', how='inner')

# 기준일 이전 값만 있는 경우 필터링
step_valid = merged_df.groupby('리뷰어').filter(lambda x: x['날짜'].max() <= cutoff_date)

# 최신 날짜의 행 선택
step_df = step_valid.loc[step_valid.groupby('리뷰어')['날짜'].idxmax()]

# 중복 제거 후 결과 저장
step_df = step_df.drop_duplicates()
step_df.to_excel(output_file, index=False)

print(f"결과가 {output_file}에 저장되었습니다.")


결과가 \Users\kyn03\Downloads\이탈자_리뷰_비공개_제거.xlsx에 저장되었습니다.


In [ ]:
base_file에서 '리뷰어'칼럼 값 중 같은 리뷰어가 3번 이상 등장할 때, 해당 행을 모두 append.

In [1]:
import pandas as pd

# 파일 경로 설정
base_file = r"\Users\kyn03\OneDrive\바탕 화면\project_file\달바_결측치처리_최종.xlsx"
output_file = r"\Users\kyn03\Downloads\리뷰어_3회이상.xlsx"

# 데이터 읽기
base_df = pd.read_excel(base_file)

# 리뷰어별 등장 횟수 계산
reviewer_counts = base_df['리뷰어'].value_counts()

# 3번 이상 등장한 리뷰어 필터링
reviewers_with_3_or_more = reviewer_counts[reviewer_counts >= 3].index

# 해당 리뷰어의 행만 선택
filtered_df = base_df[base_df['리뷰어'].isin(reviewers_with_3_or_more)]

# 결과 저장
filtered_df.to_excel(output_file, index=False)

print(f"3번 이상 등장한 리뷰어의 행이 {output_file}에 저장되었습니다.")


3번 이상 등장한 리뷰어의 행이 \Users\kyn03\Downloads\리뷰어_3회이상.xlsx에 저장되었습니다.
